<a href="https://colab.research.google.com/github/AlbezJelt/compass-aligned-graph-embeddings/blob/main/notebooks/Cade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install folium==0.2.1
!pip install git+https://github.com/valedica/gensim.git
!pip install -U cade

In [2]:
%%capture
#download file.txt
!wget https://raw.githubusercontent.com/AlbezJelt/compass-aligned-graph-embeddings/main/data/wiki_walks_from_dbpedia.txt
!wget https://raw.githubusercontent.com/AlbezJelt/compass-aligned-graph-embeddings/main/data/wikidata_walks_final.txt
#compass.txt for cade
!cat wiki_walks_from_dbpedia.txt wikidata_walks_final.txt >> compass.txt

In [3]:
import json
import requests
from cade.cade import CADE
from gensim.models.word2vec import Word2Vec
from scipy.spatial.distance import cosine

In [4]:
#load dictionary with labels
with requests.get("https://raw.githubusercontent.com/AlbezJelt/compass-aligned-graph-embeddings/main/data/wikidata_label_dictionary.json", "rt") as req:
  label_dict = json.loads(req.text)

In [5]:
#train compass
aligner = CADE(size=30)
aligner.train_compass("compass.txt", overwrite=False)

Training the compass from scratch.


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
#train 2 slices on texts
slice_one = aligner.train_slice("wiki_walks_from_dbpedia.txt", save=True)
slice_two = aligner.train_slice("wikidata_walks_final.txt", save=True)

Training embeddings: slice wiki_walks_from_dbpedia.txt.


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Initializing embeddings from compass.
Training embeddings: slice wikidata_walks_final.txt.
Initializing embeddings from compass.


In [7]:
#create models for texts
model1 = Word2Vec.load("model/wiki_walks_from_dbpedia.model")
model2 = Word2Vec.load("model/wikidata_walks_final.model")

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [72]:
def graph_creation(startpoint, epoch):
  if epoch == 0:
    return 0
  elif startpoint in model1.wv.vocab:
    similar = model2.wv.similar_by_vector(model1[startpoint])[slice(0,3)]
    if startpoint not in G:
      G.add_node(label_dict[startpoint])
    for i in range (0,3):
      word = label_dict[similar[i][0]]
      weight = similar[i][1]
      edge = (label_dict[startpoint], word, {"weight" : weight})
      if word not in G:
        G.add_node(word)
      if not(G.has_edge(*edge[:2])):
        G.add_edge(*edge[:2])
    graph_creation(similar[0][0], epoch-1)
    graph_creation(similar[1][0], epoch-1)
    graph_creation(similar[2][0], epoch-1)
    return 0
  else:
    return 0

In [75]:
import networkx as nx
G=nx.DiGraph()
startpoint = "Q38"
graph_creation(startpoint, 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


0

In [55]:
%%capture
!pip install pyvis

In [76]:
from pyvis.network import Network
net = Network(notebook=True)
net.from_nx(G)
net.show("example.html")



In [67]:
 similar = model2.wv.similar_by_vector(model1[startpoint])[slice(0,3)]
 label_dict[similar[2][0]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


'Category:Deaths in Friuli–Venezia Giulia'